##Wikipedia API

#To Do

1. Find a way to efficiently scan the corpus for articles that relate to user's concept

###Alexander's Notes

We should be piping the user's input directly into Concept Expansion
build a list of related terms, then piping that list into Concept Insight
in order to collect conceptual context (insight) about terms in that list. 

In [1]:
from wikipedia import Wikipedia
from wiki2plain import Wiki2Plain
import numpy as np

In [2]:
import ast
import json
import requests

In [3]:
# print_function turns print into a python 3 function 
# this enables python to print funky text i.e. f\xn --> fun
from __future__ import (division,
                        absolute_import,
                        print_function,
                        unicode_literals
                        )

##Corpus Curation - Wiki

####Get Math Topics from Wiki

In [4]:
headers = {'user_agent': 'DataWrangling/1.1 (http://galvanize.it; class@galvanize.it)'}
page = 'Lists_of_mathematics_topics'
payload = 'action=parse&format=json&page=' + page
url = "https://en.wikipedia.org/w/api.php?"
r = requests.post(url, data=payload, headers=headers)
topic_pages = r.json()['parse']['links']

corpus_pages = []
for i,page_complex in enumerate(topic_pages):
    # This list will be what we use to scrape, so if it is just a "list of" page
    # we are going to skip it for now
    if not page_complex['*'].startswith(u'List'):
        payload = 'action=parse&format=json&page=' + page_complex['*']
        url = "https://en.wikipedia.org/w/api.php?"
        try:
            r = requests.post(url, data=payload, headers=headers)
            corpus_pages.extend(r.json()['parse']['links'])
        except UnicodeEncodeError:
            print('Error getting: ' + page_complex['*'])
            continue
        if i % 10 == 0 or i == len(topic_pages):
            print('Processing ' + str(i) + ' of ' + str(len(topic_pages)))
        
print('Total scrapped pages: ' + str(len(corpus_pages)))

Processing 0 of 384
Processing 10 of 384
Processing 20 of 384
Processing 30 of 384
Error getting: Erdős conjecture (disambiguation)
Processing 40 of 384
Processing 50 of 384
Processing 60 of 384
Processing 300 of 384
Processing 310 of 384
Error getting: Paul Erdős
Processing 330 of 384
Processing 340 of 384
Error getting: Wikipedia:WikiProject Mathematics/List of mathematics articles (0–9)
Processing 360 of 384
Processing 370 of 384
Processing 380 of 384
Total scrapped pages: 78766


In [6]:
subject

u'List of mathematicians (E)'

In [ ]:
# upload file to corpus
lang = 'en' # standard english wiki pages
wiki = Wikipedia(lang)
for i, page in enumerate(corpus_pages):
    try: 
        subject = page["*"]
        raw = wiki.article(subject)
        wiki2plain = Wiki2Plain(raw)
        content = wiki2plain.text
        url = 'https://gateway.watsonplatform.net/concept-insights/api/v2/corpora/liet4blerinu2/math_corp/documents/' + subject

    #     payload = {"label": subject,
    #                "parts": {"data": content,
    #                           "name": subject,
    #                           "content-type": "text"},
    #                "user_fields": {"myField1": "my meta-data"}}


        payload = {"label": subject,
               "parts": [{"data": content,
                          "name": "article",
                          "content-type": "text"},
                         {"data": "Part of test document pure text."
                          ,"name": "Text part",
                          "content-type": "text/plain"}],
               "user_fields": {"myField1": "my meta-data"}}

        r2 = requests.post(url = url,
                          data = json.dumps(payload),
                          auth=auth)
    except:
        print ("some error", "file", i,  subject)
        
    if r2.status_code != 201:
        print (r2.status_code, subject)
    if i % 10 == 0:
        print (i)

0
10
20
30
some error file 31 Wikipedia:WikiProject Mathematics/List of mathematics articles (0–9)
400 Wikipedia:WikiProject Mathematics/List of mathematics articles (A)
400 Wikipedia:WikiProject Mathematics/List of mathematics articles (B)
400 Wikipedia:WikiProject Mathematics/List of mathematics articles (C)
400 Wikipedia:WikiProject Mathematics/List of mathematics articles (D)
400 Wikipedia:WikiProject Mathematics/List of mathematics articles (E)
400 Wikipedia:WikiProject Mathematics/List of mathematics articles (F)
400 Wikipedia:WikiProject Mathematics/List of mathematics articles (G)
400 Wikipedia:WikiProject Mathematics/List of mathematics articles (H)
400 Wikipedia:WikiProject Mathematics/List of mathematics articles (I)
40
400 Wikipedia:WikiProject Mathematics/List of mathematics articles (J)
400 Wikipedia:WikiProject Mathematics/List of mathematics articles (K)
400 Wikipedia:WikiProject Mathematics/List of mathematics articles (L)
400 Wikipedia:WikiProject Mathematics/List of 

In [115]:
# Now we need to pass each of these into Xander's code.
corpus_pages[700:710][0]

{u'*': u'Circle', u'exists': u'', u'ns': 0}

In [40]:
subjects = []
for page in corpus_pages[700:710]:
    print(page["*"])
    subjects.append(page["*"])

Circle
Circumference
Closure operator
Combinatorics
Commutative algebra
Compass-and-straightedge construction
Complement (set theory)
Complex analysis
Complex geometry
Complex manifold


In [14]:
import os.path

def scrape_wiki(path,topic):
    '''
    Get all text from wiki page based on the topic entered.
    '''
    lang = 'en' # standard english wiki pages
    wiki = Wikipedia(lang)
        
    fname = path + topic + '.txt'
    
    if not os.path.isfile(fname):    
        try:
            raw = wiki.article(topic)
        except:
            raw = None

        if raw:
            wiki2plain = Wiki2Plain(raw)
            content = wiki2plain.text

            with open(fname,'w') as handle:
                handle.write(content) 

In [68]:
path = "./data/"
for subject in subjects:
    scrape_wiki(path,subject)

In [69]:
f = open(path + subjects[0] + '.txt','r')

In [70]:
print(f.read())

A circle is a simple shape in Euclidean geometry. It is the set of all points in a plane that are at a given distance from a given point, the centre; equivalently it is the curve traced out by a point that moves so that its distance from a given point is constant. The distance between any of the points and the centre is called the radius.

A circle is a simple closed curve which divides the plane into two regions: an interior and an exterior. In everyday use, the term "circle" may be used interchangeably to refer to either the boundary of the figure, or to the whole figure including its interior; in strict technical usage, the circle is only the boundary and the whole figure is called a disk.

A circle may also be defined as a special ellipse in which the two foci are coincident and the eccentricity is 0, or the two-dimensional shape enclosing the most area per unit perimeter squared, using calculus of variations.

==Terminology==
*Arc: any connected part of the circle.
*Centre: the po

##Concept Insights API

##Build Corpus

In [8]:
# the ConceptInsights api from watson_developer_cloud is difficult to work with
# so I've been using request instead
from watson_developer_cloud import ConceptInsightsV2 as ConceptInsights
from pprint import pprint

In [9]:
# get account_id
base_url = "https://watson-api-explorer.mybluemix.net/concept-insights/api"
account_url = 'https://watson-api-explorer.mybluemix.net/concept-insights/api/v2/accounts'
auth=('82a698f0-8d8c-42f9-9c7d-1e4b97eebd52','50KJG6jdgZ6h')

r = requests.get(account_url, auth=auth)

In [10]:
# display account id
print(r.content)

{"accounts":[{"account_id":"liet4blerinu2"}]}


In [7]:
# GET /v2/corpora/{account_id}/{corpus}/documents/{document}/annotations
url = 'https://gateway.watsonplatform.net/concept-insights/api/v2/corpora/liet4blerinu2/math_corp/documents/Calculus/annotations'
r = requests.get(url,auth=auth)

print (r.status_code)

ast.literal_eval(r.content)

200


{'annotations': [[{'concept': {'id': '/graphs/wikipedia/en-20120601/concepts/List_of_calculus_topics',
     'label': 'List of calculus topics'},
    'parts_index': 0,
    'score': 0.98723805,
    'text_index': [31244, 31267]},
   {'concept': {'id': '/graphs/wikipedia/en-20120601/concepts/Antiderivative',
     'label': 'Antiderivative'},
    'parts_index': 0,
    'score': 0.9855932,
    'text_index': [24341, 24356]},
   {'concept': {'id': '/graphs/wikipedia/en-20120601/concepts/Antiderivative',
     'label': 'Antiderivative'},
    'parts_index': 0,
    'score': 0.98472995,
    'text_index': [24112, 24126]},
   {'concept': {'id': '/graphs/wikipedia/en-20120601/concepts/Antiderivative',
     'label': 'Antiderivative'},
    'parts_index': 0,
    'score': 0.9840046,
    'text_index': [24423, 24437]},
   {'concept': {'id': '/graphs/wikipedia/en-20120601/concepts/Antiderivative',
     'label': 'Antiderivative'},
    'parts_index': 0,
    'score': 0.98394763,
    'text_index': [24089, 24108]},

In [9]:
24231 -24200  == len('Fundamental theorem of calculus')

True

In [56]:
# create copus called 'math'
# I had a very hard time trying to get this code to work - hours lost ! 
# It still doesn't work. Maybe I should be using '.post' instead of '.put' ...?

# create_corp = "https://gateway.watsonplatform.net/concept-insights/api/v2/corpora/liet4blerinu2/math"
# payload = '{"access":"private","users":[{"uid":"82a698f0-8d8c-42f9-9c7d-1e4b97eebd52","permission":"AdminReadWrite"}]}'
# r2 = requests.put(create_corp,
#                   params = payload,
#                   auth=auth)
# r2.status_code

400

In [10]:
# create corpus called 'math_corp'
# this code only needs to run once -- otherwise 409 error will occur

!curl -D - -X PUT -u "82a698f0-8d8c-42f9-9c7d-1e4b97eebd52":'50KJG6jdgZ6h' \
    -d  '{"access":"private","users":[{"uid":"82a698f0-8d8c-42f9-9c7d-1e4b97eebd52","permission":"AdminReadWrite"}]}'\
    "https://gateway.watsonplatform.net/concept-insights/api/v2/corpora/liet4blerinu2/math_corp"














{"error":"corpus already exists","url":"/api/v2/corpora/liet4blerinu2/math_corp"}


In [74]:
# get available corpora
available_corpora = 'https://gateway.watsonplatform.net/concept-insights/api/v2/corpora/liet4blerinu2'

r = requests.get(available_corpora, auth=auth)
r.status_code

200

In [75]:
# display available corpora and it's parameters
ast.literal_eval(r.content)

{'corpora': [{'access': 'private',
   'id': '/corpora/liet4blerinu2/math_corp',
   'users': [{'account_id': '', 'permission': 'ReadWriteAdmin'},
    {'account_id': 'liet4blerinu2', 'permission': 'ReadWriteAdmin'}]}]}

In [18]:
# DELETE /v2/corpora/{account_id}/{corpus} Deletes a corpus by ID
url = 'https://gateway.watsonplatform.net/concept-insights/api/v2/corpora/liet4blerinu2/math_corp2'
requests.delete(url)

<Response [401]>

In [22]:
# DELETE /v2/corpora/{account_id}/{corpus} Deletes a corpus by ID
!curl -u "82a698f0-8d8c-42f9-9c7d-1e4b97eebd52":'50KJG6jdgZ6h' \
        -X DELETE 'https://gateway.watsonplatform.net/concept-insights/api/v2/corpora/liet4blerinu2/math_corp2'

In [76]:
# upload file to corpus
url = 'https://gateway.watsonplatform.net/concept-insights/api/v2/corpora/liet4blerinu2/math_corp/documents/testDocument'
path ="data/"
for subject in subjects:
    url = 'https://gateway.watsonplatform.net/concept-insights/api/v2/corpora/liet4blerinu2/math_corp/documents/' + subject

    payload = {"label": subject,
               "parts": [{"data": open(path + subject + '.txt','rb').read(),
                          "name": "HTML part",
                          "content-type": "text/html"},
                         {"data": "Part of test document pure text."
                          ,"name": "Text part",
                          "content-type": "text/plain"}],
               "user_fields": {"myField1": "my meta-data"}}

    r2 = requests.post(url = url,
                      data = json.dumps(payload),
                      auth=auth)
    print(r2.status_code)

201
201
201
201
201
201
201
201
201
201


### retrive document id's

In [77]:
# GET /v2/corpora/{account_id}/{corpus}/documents
url = 'https://gateway.watsonplatform.net/concept-insights/api/v2/corpora/liet4blerinu2/math_corp/documents'
r = requests.get(url,auth=auth)

print (r.status_code)

200


In [78]:
ast.literal_eval(r.content)

{'documents': ['/corpora/liet4blerinu2/math_corp/documents/Calculus',
  '/corpora/liet4blerinu2/math_corp/documents/Trigonometry',
  '/corpora/liet4blerinu2/math_corp/documents/Geometry',
  '/corpora/liet4blerinu2/math_corp/documents/Differential%20Equations',
  '/corpora/liet4blerinu2/math_corp/documents/Integrals',
  '/corpora/liet4blerinu2/math_corp/documents/Unit%20Circle',
  '/corpora/liet4blerinu2/math_corp/documents/Sine',
  '/corpora/liet4blerinu2/math_corp/documents/Cosine',
  '/corpora/liet4blerinu2/math_corp/documents/pythagorean%20theorem',
  '/corpora/liet4blerinu2/math_corp/documents/Circle',
  '/corpora/liet4blerinu2/math_corp/documents/Circumference',
  '/corpora/liet4blerinu2/math_corp/documents/Closure%20operator',
  '/corpora/liet4blerinu2/math_corp/documents/Combinatorics',
  '/corpora/liet4blerinu2/math_corp/documents/Commutative%20algebra',
  '/corpora/liet4blerinu2/math_corp/documents/Compass-and-straightedge%20construction',
  '/corpora/liet4blerinu2/math_corp/d

In [79]:
# get content of document
url = 'https://gateway.watsonplatform.net/concept-insights/api/v2/corpora/liet4blerinu2/math_corp/documents/Complex%20manifold'
r = requests.get(url,auth=auth)

print (r.status_code)

ast.literal_eval(r.content)

200


{'id': '/corpora/liet4blerinu2/math_corp/documents/Complex%20manifold',
 'label': 'Complex manifold',
 'last_modified': '2015-12-05T01:27:36.359Z',
 'parts': [{'content-type': 'text/html',
   'data': 'In differential geometry, a complex manifold is a manifold with an atlas of charts to the open unit diskOne must use the open unit disk in Cn as the model space instead of Cn because these are not isomorphic, unlike for real manifolds. in Cn, such that the transition maps are holomorphic.\n\nThe term complex manifold is variously used to mean a complex manifold in the sense above (which can be specified as an integrable complex manifold), and an almost complex manifold.\n\n==Implications of complex structure==\nSince holomorphic functions are much more rigid than smooth functions, the theories of smooth and complex manifolds have very different flavors: compact complex manifolds are much closer to algebraic varieties than to differentiable manifolds.\n\nFor example, the Whitney embedding 

In [29]:
# delete document from corpus by id
# DELETE /v2/corpora/{account_id}/{corpus}/documents/{document}

! curl -u "82a698f0-8d8c-42f9-9c7d-1e4b97eebd52":'50KJG6jdgZ6h'\
       -X DELETE 'https://gateway.watsonplatform.net/concept-insights/api/v2/corpora/liet4blerinu2/math_corp/documents/testDocument'

### stats on concepts present in corpora 

In [80]:
# GET /v2/corpora/{account_id}/{corpus}/stats
url = 'https://gateway.watsonplatform.net/concept-insights/api/v2/corpora/liet4blerinu2/math_corp/stats'
r = requests.get(url,auth=auth)
r.status_code

200

In [81]:
# stats on concepts present in corpora 
ast.literal_eval(r.content)

{'id': '/corpora/liet4blerinu2/math_corp',
 'last_updated': '2015-12-05T01:29:17.296Z',
 'top_tags': {'corpus_tags_histogram': {'1': 1173,
   '10': 21,
   '11': 18,
   '111': 1,
   '112': 1,
   '115': 1,
   '117': 1,
   '12': 19,
   '13': 18,
   '136': 1,
   '14': 13,
   '147': 1,
   '15': 6,
   '158': 1,
   '16': 11,
   '160': 1,
   '166': 1,
   '168': 1,
   '17': 10,
   '178': 1,
   '18': 4,
   '181': 1,
   '186': 1,
   '19': 11,
   '199': 1,
   '2': 391,
   '20': 4,
   '21': 4,
   '22': 6,
   '225': 1,
   '23': 2,
   '24': 1,
   '25': 3,
   '26': 2,
   '27': 6,
   '28': 1,
   '29': 2,
   '3': 176,
   '30': 1,
   '31': 1,
   '313': 1,
   '32': 2,
   '33': 3,
   '339': 1,
   '34': 2,
   '35': 3,
   '38': 3,
   '39': 2,
   '4': 150,
   '40': 1,
   '405': 1,
   '41': 1,
   '43': 1,
   '44': 4,
   '45': 2,
   '46': 1,
   '47': 2,
   '49': 2,
   '5': 60,
   '50': 1,
   '51': 1,
   '55': 1,
   '56': 3,
   '57': 1,
   '58': 1,
   '59': 1,
   '6': 51,
   '61': 2,
   '62': 1,
   '64': 1,
   '

###graph id

In [82]:
url = 'https://gateway.watsonplatform.net/concept-insights/api/v2/graphs'
#url = "https://watson-api-explorer.mybluemix.net/concept-insights/api/v2/graphs"
r = requests.get(url ,auth=auth)
r.status_code

200

In [83]:
ast.literal_eval(r.content)

{'graphs': ['/graphs/wikipedia/en-20120601', '/graphs/wikipedia/en-latest']}

###label search

In [96]:
# GET /v2/graphs/{account_id}/{graph}/label_search
!curl -u '82a698f0-8d8c-42f9-9c7d-1e4b97eebd52':'50KJG6jdgZ6h''https://gateway.watsonplatform.net/concept-insights/api/v2/graphs/liet4blerinu2/wikipedia/en-latest/label_search?query=math'

curl: no URL specified!
curl: try 'curl --help' or 'curl --manual' for more information


In [95]:
url = 'https://gateway.watsonplatform.net/concept-insights/api'+\
 '/v2/graphs/wikipedia/en-20120601/label_search?query=quadratic'
r = requests.get(url ,auth=auth)
r.status_code

200

In [195]:
ast.literal_eval(r.content)

{'matches': [{'id': '/graphs/wikipedia/en-20120601/concepts/Quadratic',
   'label': 'Quadratic'}]}

In [158]:
#GET /v2/corpora/{account_id}/{corpus}/conceptual_search
url = 'https://gateway.watsonplatform.net/concept-insights/api'+\
 '/v2/corpora/liet4blerinu2/math_corp/conceptual_search?concept_fields=["sine"]'
r  = requests.get(url, auth=auth)
r.status_code

400

In [159]:
r.content #"/graphs/wikipedia/en-20120601/concepts/Slope

'{"error":"missing ids parameter","url":"/api/v2/corpora/liet4blerinu2/math_corp/conceptual_search"}\n'

###Concepts related to a specific article

The course will focus on Trigonometry: sine, cosine, unit circles and the like. 

In [110]:
#GET /v2/corpora/{account_id}/{corpus}/documents/{document}
# Calculus
q = 'geometry'
url = 'https://gateway.watsonplatform.net/concept-insights/api'+\
      '/v2/corpora/liet4blerinu2/math_corp/documents/' + q + '/related_concepts'
    
# url = 'https://gateway.watsonplatform.net/concept-insights/api'+\
#       '/v2/corpora/liet4blerinu2/math_corp/documents/Complex%20manifold/related_concepts'

r  = requests.get(url, auth=auth)
r.status_code

400

In [111]:
ast.literal_eval(r.content)

{'error': 'document not found',
 'url': '/api/v2/corpora/liet4blerinu2/math_corp/documents/geometry/related_concepts'}

In [112]:
url = 'https://gateway.watsonplatform.net/concept-insights/api'+\
      '/v2/corpora/liet4blerinu2/math_corp/documents/Geometry/related_concepts'
r  = requests.get(url, auth=auth)
r.status_code

200

In [113]:
ast.literal_eval(r.content)

{'concepts': [{'concept': {'id': '/graphs/wikipedia/en-20120601/concepts/Euclid',
    'label': 'Euclid'},
   'score': 0.98175925},
  {'concept': {'id': '/graphs/wikipedia/en-20120601/concepts/Non-Euclidean_geometry',
    'label': 'Non-Euclidean geometry'},
   'score': 0.98127043},
  {'concept': {'id': '/graphs/wikipedia/en-20120601/concepts/List_of_geometers',
    'label': 'List of geometers'},
   'score': 0.9812166},
  {'concept': {'id': '/graphs/wikipedia/en-20120601/concepts/Alhazen',
    'label': 'Alhazen'},
   'score': 0.98114043},
  {'concept': {'id': '/graphs/wikipedia/en-20120601/concepts/Indian_mathematics',
    'label': 'Indian mathematics'},
   'score': 0.98109186},
  {'concept': {'id': '/graphs/wikipedia/en-20120601/concepts/Space',
    'label': 'Space'},
   'score': 0.98109114},
  {'concept': {'id': '/graphs/wikipedia/en-20120601/concepts/John_Wallis',
    'label': 'John Wallis'},
   'score': 0.9810903},
  {'concept': {'id': "/graphs/wikipedia/en-20120601/concepts/Euclid's

##How Related is Radian (Concept) to Calculus (Document)

In [97]:
# GET /v2/corpora/{account_id}/{corpus}/documents/{document}/relation_scores
# concept used has to be present in the concept graph -- otherwise error will be given
concept = "/graphs/wikipedia/en-20120601/concepts/holomorphic embedding"
url = 'https://gateway.watsonplatform.net/concept-insights/api'+\
      '/v2/corpora/liet4blerinu2/math_corp/documents/Complex%20manifold/'+\
    'relation_scores?concepts=["/graphs/wikipedia/en-20120601/concepts/"]'
r  = requests.get(url, auth=auth)
r.status_code

200

In [98]:
ast.literal_eval(r.content) # "/graphs/wikipedia/en-20120601/concepts/Radian"

{'scores': [{'concept': '/graphs/wikipedia/en-20120601/concepts/',
   'score': 0.5}]}

##ConceptExpansion API

In [7]:
import json
import time
from watson_developer_cloud import ConceptExpansionV1Beta as ConceptExpansion


concept_expansion = ConceptExpansion(username='82a698f0-8d8c-42f9-9c7d-1e4b97eebd52',
                                     password='50KJG6jdgZ6h')

job_id = concept_expansion.create_job(dataset='mtsamples', 
                                      seeds=['motrin', 'tylenol', 'aspirin'], 
                                      label='medications')
print(json.dumps(job_id, indent=2))

time.sleep(5)  # sleep for 5 seconds
job_status = concept_expansion.get_status(job_id)

while job_status['status'] == 'in progress' or job_status['status'] == 'awaiting work':
    time.sleep(5)  # sleep for 5 seconds
    job_status = concept_expansion.get_status(job_id)
    print(json.dumps(job_status, indent=2))

if job_status['status'] == 'done':
    results = concept_expansion.get_results(job_id)
    print(json.dumps(results, indent=2))

{
  "jobid": "M0hUQkdKOTZJWkNDSUhRSVVCT047MTI0MzAxMTE="
}
{
  "return_seeds": [
    {
      "result": "ibuprofen", 
      "prevalence": 0.8219226002693176
    }, 
    {
      "result": "advil", 
      "prevalence": 0.80472731590271
    }, 
    {
      "result": "naproxen", 
      "prevalence": 0.7841069102287292
    }, 
    {
      "result": "benadryl", 
      "prevalence": 0.7685651183128357
    }, 
    {
      "result": "acetaminophen", 
      "prevalence": 0.7564734220504761
    }, 
    {
      "result": "codeine", 
      "prevalence": 0.743308424949646
    }, 
    {
      "result": "pain relievers", 
      "prevalence": 0.7278230786323547
    }, 
    {
      "result": "laxatives", 
      "prevalence": 0.7256090641021729
    }, 
    {
      "result": "motrin ib", 
      "prevalence": 0.7229762077331543
    }, 
    {
      "result": "aleve", 
      "prevalence": 0.7223989367485046
    }, 
    {
      "result": "diuretics", 
      "prevalence": 0.7221292853355408
    }, 
    {
      "